In [34]:
import os
import pandas as pd
from PIL import Image
import numpy as np
import tensorflow as tf
from sklearn.model_selection import train_test_split
from keras.utils import to_categorical
from keras.optimizers import Adam
from sklearn.preprocessing import LabelBinarizer
from preprocess import preprocess_image, get_labels

os.environ['CUDA_VISIBLE_DEVICES'] = '0'


In [103]:
def preprocess_image(image_path):
    label = get_labels(image_path)

    # Read the image from file
    image_string = tf.io.read_file(image_path)
    image_decoded = tf.image.decode_jpeg(image_string)

    # Convert the image to 3-channel RGB
    image_rgb = tf.image.grayscale_to_rgb(image_decoded)

    # Resize and crop the image to a fixed size
    resized_image = tf.image.resize(image_rgb, [224, 224])

    # Normalize the pixel values of the image
    normalized_image = tf.image.per_image_standardization(resized_image)

    return normalized_image, label

In [126]:
def add_dimension(image, label):
  # Add an additional dimension to the image
  image = tf.expand_dims(image, axis=0)
  return image, label

In [127]:
data_path = 'data2/'

image_ds = tf.data.Dataset.list_files(data_path + '*/*', shuffle = 'False')

image_ds = image_ds.shuffle(200)


In [132]:
img_count = len(image_ds)


train_size = int(img_count*0.8) 

# print(image_ds.take(1))

train_ds = image_ds.take(train_size)

# print(next(iter(train_ds)).numpy())

test_ds = image_ds.skip(train_size)

train_ds = train_ds.map(preprocess_image)
test_ds = test_ds.map(preprocess_image)

# print(len(train_ds))

train_ds = train_ds.map(add_dimension)
test_ds = test_ds.map(add_dimension)



AttributeError: 'MapDataset' object has no attribute 'shape'

In [129]:
train_ds.take(1)

<TakeDataset element_spec=(TensorSpec(shape=(1, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(), dtype=tf.string, name=None))>

In [106]:
# Create a set to store the unique labels
unique_labels = set()

# Loop through the dataset and add each label to the set
# for image in train_ds:
#   print(image)
#   print(label.unique_with_counts)
#   unique_labels.add(image)

# The length of the set will be the total number of unique labels
total_labels = len(unique_labels)

In [130]:


# Define the model
model = tf.keras.Sequential()

# Add convolutional layers
model.add(tf.keras.layers.Conv2D(32, (3, 3), activation='relu', input_shape=(224, 224, 3)))
model.add(tf.keras.layers.Conv2D(64, (3, 3), activation='relu'))
model.add(tf.keras.layers.MaxPooling2D((2, 2)))

# Add a flattening layer
model.add(tf.keras.layers.Flatten())

# Add a dense
model.add(tf.keras.layers.Dense(128, activation='relu'))
model.add(tf.keras.layers.Dense(4, activation='softmax'))


In [136]:
model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])   



model.fit(train_ds, validation_data=test_ds, batch_size=32, epochs=10)

# model.save('model.h5')

Epoch 1/10


ValueError: in user code:

    File "c:\Users\thunb\Documents\Dev\Random\python\medical_ai\.env\lib\site-packages\keras\engine\training.py", line 1249, in train_function  *
        return step_function(self, iterator)
    File "c:\Users\thunb\Documents\Dev\Random\python\medical_ai\.env\lib\site-packages\keras\engine\training.py", line 1233, in step_function  **
        outputs = model.distribute_strategy.run(run_step, args=(data,))
    File "c:\Users\thunb\Documents\Dev\Random\python\medical_ai\.env\lib\site-packages\keras\engine\training.py", line 1222, in run_step  **
        outputs = model.train_step(data)
    File "c:\Users\thunb\Documents\Dev\Random\python\medical_ai\.env\lib\site-packages\keras\engine\training.py", line 1024, in train_step
        loss = self.compute_loss(x, y, y_pred, sample_weight)
    File "c:\Users\thunb\Documents\Dev\Random\python\medical_ai\.env\lib\site-packages\keras\engine\training.py", line 1082, in compute_loss
        return self.compiled_loss(
    F